In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
df = pd.read_csv('train.csv')

In [11]:
df.shape

(404290, 6)

In [12]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [13]:
new_df = df.sample(50000)
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [14]:
new_df.duplicated().sum()

0

In [15]:
ques_df = new_df[['question1','question2']]
ques_df.head(10)
new_df.head(10)

,id,qid1,qid2,question1,question2,is_duplicate
366158,366158,496332,496333,What are the best business book summaries?,What are the best business books to read?,0
341333,341333,469136,469137,Who do you think wants the ownership of the as...,Where do I go if I want to create a new blockc...,0
362655,362655,268340,345886,Which is a suitable inpatient drug and alcohol...,What is a suitable inpatient drug and alcohol ...,0
283315,283315,403400,403401,Can African Americans tell us whether they lik...,What do black Americans think of Django Unchai...,0
217016,217016,5902,1596,What are the safety precautions on handling sh...,What are the safety precautions on handling sh...,1
164312,164312,255344,255345,My card has been blocked by umobile?,What is AAFT? Which type of courses do they pr...,0
71155,71155,122499,122500,Which is the best country to work?,Which is best country to work for?,1
236535,236535,134574,146279,Can I see who viewed my profile on WhatsApp?,How do I see who stalks my whatsapp profile us...,1
237865,237865,349104,349105,Why do I want to go far away from everyone and...,Why do I want to distance myself from everyone...,1
165097,165097,256422,256423,What are some examples of neutrons and protons?,How does a neutron turn into a proton and an e...,0


In [16]:
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])
cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [17]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(50000, 6000)

In [18]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [19]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
366158,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
341333,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
362655,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
283315,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
217016,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [20]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2,random_state=1)

In [21]:
X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7526